# Train RotatE Models on standardized datasets using published hyperparameters
given prior published parameters, implement them in PyKEEN for WN18, WN18RR, FB15k & FB15k-237

## RotatE Hyperparameters

* The published parameters can be found [here](https://github.com/DeepGraphLearning/KnowledgeGraphEmbedding/blob/master/best_config.sh) and are shown below
```bash

#             1     2     3    4 5   6   7   8    9    10  11    12     13  14
bash run.sh train RotatE FB15k 0 0 1024 256 1000 24.0 1.0 0.0001 150000 16 -de
bash run.sh train RotatE FB15k-237 0 0 1024 256 1000 9.0 1.0 0.00005 100000 16 -de
bash run.sh train RotatE wn18 0 0 512 1024 500 12.0 0.5 0.0001 80000 8 -de
bash run.sh train RotatE wn18rr 0 0 512 1024 500 6.0 0.5 0.00005 80000 8 -de
```

* The format is as follow:
```bash
MODE=$1
MODEL=$2
DATASET=$3
GPU_DEVICE=$4
SAVE_ID=$5

FULL_DATA_PATH=$DATA_PATH/$DATASET
SAVE=$SAVE_PATH/"$MODEL"_"$DATASET"_"$SAVE_ID"

#Only used in training
BATCH_SIZE=$6
NEGATIVE_SAMPLE_SIZE=$7 # keep in mind, this is not a sampling ratio
HIDDEN_DIM=$8
GAMMA=$9
ALPHA=${10}
LEARNING_RATE=${11}
MAX_STEPS=${12}
TEST_BATCH_SIZE=${13}
```

# Train a RotatE model using PyKEEN

## Train a RotatE model on FB15k

```bash
#             1     2     3    4 5   6   7   8    9    10  11    12     13  14
bash run.sh train RotatE FB15k 0 0 1024 256 1000 24.0 1.0 0.0001 150000 16 -de
```

In [1]:
import pykeen
import pykeen.datasets as pkd
import pykeen.pipeline as pkp
import sys
import os
import torch
import torch.nn
import numpy as np
import torch.nn.functional as F

In [2]:
model_kwargs = dict(
    # Model
    model="RotatE",
    model_kwargs=dict(embedding_dim=1000),
    # Loss
    loss="NSSALoss",
    loss_kwargs=dict(
        margin=24.0,  # gamma
        adversarial_temperature=1.0,  # alpha
    ),
    # Regularization
    # regularizer="LpRegularizer",
    # regularizer_kwargs=dict(weight=), # No regularization
    # Training
    training_kwargs=dict(
        num_epochs=317,  # 592,213 Triples, 150,000 iterations, with batch sz of 1024 means 259 epochs
        # however, for some reason, each epoch is 472 iterations, so 150,000/472 = 317 epochs
        batch_size=1024,
        checkpoint_frequency=0,
        checkpoint_name="RotatE_FB15k.pt",
    ),
    # Negative Sampler
    negative_sampler="basic",
    negative_sampler_kwargs=dict(
        num_negs_per_pos=256,  # corruption_scheme=("h","r","t",),  # defines which part of the triple to corrupt
        filtered=True,  # Uses a default 'Bloom' filter to minimize false negatives
    ),
    # optimizer
    optimizer="Adam",
    optimizer_kwargs=dict(lr=0.0001),
    # lr scheduler
    lr_scheduler="StepLR",
    lr_scheduler_kwargs=dict(
        gamma=0.1, step_size=158
    ),  # 1/10 the learning rate every 158 epochs
    # earlystopper
    # stopper='early',
    # stopper_kwargs=dict(patience = 1, relative_delta = 0.00005,), # no frequency var, otherwise it forces an eval at specified epoch
    # Tracking
    result_tracker="wandb",
    result_tracker_kwargs=dict(project="KGE-on-time-results", group="FB15k"),
    # Misc
    device="cuda:1",  # use gpu position 1
)

In [5]:
dataset = pkd.get_dataset(dataset="FB15k")
res = pkp.pipeline(dataset=dataset, **model_kwargs)

In [ ]:
del res
torch.cuda.empty_cache()

## Train a model on FB15k-237

```bash
#             1     2     3        4 5   6   7   8    9    10  11    12     13 
bash run.sh train RotatE FB15k-237 0 0 1024 256 1000 9.0 1.0 0.00005 100000 16 -de
```

### import dataset

In [11]:
dataset = pkd.get_dataset(dataset="FB15k-237")

INFO:pykeen.datasets.utils:Loading cached preprocessed dataset from file:///home/rogertu/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///home/rogertu/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
/home/rogertu/miniforge3/envs/mini_semmed/lib/python3.12/site-packages/pykeen/triples/triples_factory.py:763: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

In [15]:
dataset.training.num_triples

272115

In [17]:
f"Number of Epochs: {100000 /(dataset.training.num_triples/1024)}"

'Number of Epochs: 376.31148595263033'

In [18]:
model_kwargs = dict(
    # Model
    model="RotatE",
    model_kwargs=dict(embedding_dim=1000),
    # Loss
    loss="NSSALoss",
    loss_kwargs=dict(
        margin=9.0,  # gamma
        adversarial_temperature=1.0,  # alpha
    ),
    # Regularization
    # regularizer="LpRegularizer",
    # regularizer_kwargs=dict(weight=), # No regularization
    # Training
    training_kwargs=dict(
        num_epochs=376,  # 272,115 training triples, 100,000 iterations, with batch sz of 1024 means 376 epochs
        batch_size=1024,
        checkpoint_frequency=0,
        checkpoint_name="RotatE_FB15k237.pt",
    ),
    # Negative Sampler
    negative_sampler="basic",
    negative_sampler_kwargs=dict(
        num_negs_per_pos=256,  # corruption_scheme=("h","r","t",),  # defines which part of the triple to corrupt
        filtered=True,  # Uses a default 'Bloom' filter to minimize false negatives
    ),
    # optimizer
    optimizer="Adam",
    optimizer_kwargs=dict(lr=0.00005),
    # lr scheduler
    lr_scheduler="StepLR",
    lr_scheduler_kwargs=dict(
        gamma=0.1, step_size=188
    ),  # 1/10 the learning rate every 188 epochs
    # earlystopper
    # stopper='early',
    # stopper_kwargs=dict(patience = 1, relative_delta = 0.00005,), # no frequency var, otherwise it forces an eval at specified epoch
    # Tracking
    result_tracker="wandb",
    result_tracker_kwargs=dict(
        project="KGE-on-time-results", group="FB15k-237", id="RotatE-FB15k237"
    ),
    # Misc
    device="cuda:1",  # use gpu position 1
)

In [21]:
res = pkp.pipeline(dataset=dataset, **model_kwargs)

In [25]:
del res
torch.cuda.empty_cache()

## Train model on WN18

```bash
#             1     2     3   4 5   6   7   8    9   10   11    12   13 14
bash run.sh train RotatE wn18 0 0 512 1024 500 12.0 0.5 0.0001 80000 8 -de
```

In [2]:
dataset = pkd.get_dataset(dataset="WN18")

/home/rogertu/miniforge3/envs/mini_semmed/lib/python3.12/site-packages/pykeen/triples/triples_factory.py:763: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = dict(torch

In [3]:
f"Epoch Size: {80000/(dataset.training.num_triples/512)}"

'Epoch Size: 289.5886653186465'

In [4]:
model_kwargs = dict(
    # Model
    model="RotatE",
    model_kwargs=dict(embedding_dim=500),
    # Loss
    loss="NSSALoss",
    loss_kwargs=dict(
        margin=12.0,  # gamma
        adversarial_temperature=0.5,  # alpha
    ),
    # Regularization
    # regularizer="LpRegularizer",
    # regularizer_kwargs=dict(weight=), # No regularization
    # Training
    training_kwargs=dict(
        num_epochs=290,
        batch_size=512,
        checkpoint_frequency=0,
        checkpoint_name="RotatE_WN18.pt",
    ),
    # Negative Sampler
    negative_sampler="basic",
    negative_sampler_kwargs=dict(
        num_negs_per_pos=1024,  # corruption_scheme=("h","r","t",),  # defines which part of the triple to corrupt
        filtered=True,  # Uses a default 'Bloom' filter to minimize false negatives
    ),
    # optimizer
    optimizer="Adam",
    optimizer_kwargs=dict(lr=0.0001),
    # lr scheduler
    lr_scheduler="StepLR",
    lr_scheduler_kwargs=dict(
        gamma=0.1, step_size=145
    ),  # 1/10 the learning rate every 145 epochs
    # earlystopper
    # stopper='early',
    # stopper_kwargs=dict(patience = 1, relative_delta = 0.00005,), # no frequency var, otherwise it forces an eval at specified epoch
    # Tracking
    result_tracker="wandb",
    result_tracker_kwargs=dict(
        project="KGE-on-time-results", group="WN18", id="RotatE-WN18"
    ),
    # Misc
    device="cuda:1",  # use gpu position 1
)

In [ ]:
res = pkp.pipeline(dataset=dataset, **model_kwargs)

In [ ]:
del res
torch.cuda.empty_cache()

## Train model on WN18RR
```bash
#             1     2     3    4 5   6   7   8    9    10  11    12     13  14
bash run.sh train RotatE wn18rr 0 0 512 1024 500 6.0 0.5 0.00005 80000 8 -de
```

In [2]:
dataset = pkd.get_dataset(dataset="WN18RR")

You're trying to map triples with 212 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3134 triples were filtered out
You're trying to map triples with 211 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3034 triples were filtered out


In [4]:
f"Epoch Size: {80000/(dataset.training.num_triples/512)}"

'Epoch Size: 471.699199631485'

In [5]:
model_kwargs = dict(
    # Model
    model="RotatE",
    model_kwargs=dict(embedding_dim=500),
    # Loss
    loss="NSSALoss",
    loss_kwargs=dict(
        margin=6.0,  # gamma
        adversarial_temperature=0.5,  # alpha
    ),
    # Regularization
    # regularizer="LpRegularizer",
    # regularizer_kwargs=dict(weight=), # No regularization
    # Training
    training_kwargs=dict(
        num_epochs=472,
        # however, for some reason, each epoch is 472 iterations, so 150,000/472 = 317 epochs
        batch_size=512,
        checkpoint_frequency=0,
        checkpoint_name="RotatE_WN18RR.pt",
    ),
    # Negative Sampler
    negative_sampler="basic",
    negative_sampler_kwargs=dict(
        num_negs_per_pos=1024,  # corruption_scheme=("h","r","t",),  # defines which part of the triple to corrupt
        filtered=True,  # Uses a default 'Bloom' filter to minimize false negatives
    ),
    # optimizer
    optimizer="Adam",
    optimizer_kwargs=dict(lr=0.00005),
    # lr scheduler
    lr_scheduler="StepLR",
    lr_scheduler_kwargs=dict(gamma=0.1, step_size=236),  # 1/10 the learning rate
    # Tracking
    result_tracker="wandb",
    result_tracker_kwargs=dict(
        project="KGE-on-time-results", group="WN18RR", id="RotatE-WN18RR"
    ),
    # Misc
    device="cuda:1",  # use gpu position 1
)

In [ ]:
res = pkp.pipeline(dataset=dataset, **model_kwargs)

In [7]:
del res
torch.cuda.empty_cache()